In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
# For finding and loading multiple files
import os
# Use this to see how much memory the dataframes use
from sys import getsizeof
# To view keyspace.yaml
import yaml
from pathlib import Path

import pseudopeople as psp

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import alpha

!date
!whoami
!uname -a
!python --version
!pwd

Thu 10 Aug 2023 05:36:26 PM PDT
ndbs
Linux long-slurm-sarchive-p0046 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
Python 3.10.12
/mnt/share/code/ndbs/vivarium_research_prl/model_validation


In [18]:
%load_ext autoreload
%autoreload 2

# Data!
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop /results/release_02_yellow/full_data/united_states_of_america/2023_07_28_08_33_09
/final_results/2023_08_02_11_38_10/pseudopeople_input_data_usa
```

In [4]:
project_dir = Path('/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop')
full_data_dir = project_dir / 'results/release_02_yellow/full_data'
usa_model_dir = full_data_dir / 'united_states_of_america/2023_07_28_08_33_09'
usa_data_dir = usa_model_dir / 'final_results/2023_08_02_11_38_10/pseudopeople_input_data_usa'

!ls -halt $usa_data_dir

total 384K
-rw-r--r--  1 albrja IHME-Simulationscience 1.6K Aug  2 15:31 CHANGELOG.rst
drwxrwsr-x 11 albrja IHME-Simulationscience 5.0K Aug  2 15:31 .
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:30 taxes_dependents
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:28 taxes_1040
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:23 taxes_w2_and_1099
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 social_security
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 women_infants_and_children
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 current_population_survey
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 american_community_survey
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 decennial_census
drwxrwsr-x  2 albrja IHME-Simulationscience 334K Aug  2 14:28 logs
drwxrwsr-x  3 albrja IHME-Simulationscience  512 Aug  2 11:38 ..


# Read `keyspace.yaml` to see seed numbers

In [7]:
!ls -halt $usa_model_dir

total 1.2M
drwxrwsr-x  5 sbachmei IHME-Simulationscience 2.5K Aug  7 18:41 ..
drwxrwsr-x  6 sbachmei IHME-Simulationscience 3.0K Aug  7 18:41 final_results
drwxrwsr-x  6 sbachmei IHME-Simulationscience 5.0K Jul 31 08:59 .
-rw-rw-r--  1 sbachmei IHME-Simulationscience 1.2M Jul 29 06:01 output.hdf
drwxrwsr-x 10 sbachmei IHME-Simulationscience 4.0K Jul 28 17:09 raw_results
drwxrwsr-x  2 sbachmei IHME-Simulationscience  512 Jul 28 08:33 __pycache__
-rw-rw-r--  1 sbachmei IHME-Simulationscience  110 Jul 28 08:33 settings.py
-rw-rw-r--  1 sbachmei IHME-Simulationscience 1.9K Jul 28 08:33 model_specification.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience   50 Jul 28 08:33 branches.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience 2.4K Jul 28 08:33 keyspace.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience 4.2K Jul 28 08:33 requirements.txt
drwxrwsr-x  3 sbachmei IHME-Simulationscience  512 Jul 28 08:33 logs


In [9]:
with open(usa_model_dir / "keyspace.yaml", 'r') as stream:
    keyspace = yaml.safe_load(stream)
keyspace.keys()

dict_keys(['input_draw', 'placeholder_branch_name.scenario', 'random_seed'])

In [13]:
' '.join(map(str, keyspace['random_seed']))

'4344 5616 6810 2787 2284 4369 3254 7359 3541 5020 9672 8869 3167 6817 4507 9901 5949 6545 1362 781 7594 8641 6144 9911 4941 5439 3947 4938 8516 6991 4561 432 4637 3793 860 323 3380 3938 107 3595 7985 494 2476 6539 5860 6800 3978 5703 3984 7438 2047 6324 7384 7817 9526 7275 4922 3465 6606 8877 4622 5548 3326 6007 1235 1061 3298 7107 8291 7551 8133 9434 9159 771 4950 3591 352 9859 93 9549 1490 1219 5188 3377 558 1614 4123 9577 7938 2500 7179 2599 1534 386 1935 338 6520 3585 7745 539 4943 4672 4743 3481 5685 5950 7054 103 5901 3142 3994 6760 1282 8360 1177 1091 5399 5308 8734 8156 6701 3390 5619 5812 3598 3357 3374 3545 7530 1597 5475 9187 5969 4400 1299 974 131 3776 7164 440 2054 4255 2405 6776 2416 6090 7137 9284 3731 3202 9588 8838 478 3454 5300 5863 6554 6442 2700 3621 1356 821 9840 9872 3568 3240 3713 1667 1508 6123 6455 1897 8817 9696 9225 6519 9804 3825 5393 9768 9495 1069 7342 2689 9024 6975 5670 4649 8134 1452 40 8612 6207 2965 1154 8997 6203 1832 3528 9871 28 3725 3304 6654 480

In [15]:
with open(usa_model_dir / "branches.yaml", 'r') as stream:
    branches = yaml.safe_load(stream)
branches # Hmm, it's a list not a dict. Why?

[{'placeholder_branch_name': {'scenario': 'baseline'}}]

In [16]:
# Looks like the single key is stored as a list item with -
!head $usa_model_dir/branches.yaml

- placeholder_branch_name:
    scenario: baseline
